### Import Libraries

In [9]:
import gym
import gym.spaces
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

### Create  game environment and examine observation

In [4]:
env = gym.make("CartPole-v0") #choose the game

obs = env.reset()
# plt.figure()
# plt.imshow(obs)
# plt.colorbar()
# plt.grid(False)
np.shape(obs)

(4,)

In [5]:
obs

array([ 0.00599583,  0.03632876,  0.03036732, -0.00559476])

### Data loading

In [7]:
env.step(1)

(array([ 0.0067224 ,  0.23100234,  0.03025542, -0.28854387]), 1.0, False, {})

In [31]:
print(env.action_space, env.observation_space)

Discrete(2) Box(4,)


In [ ]:
n_actions = env.action_space.n #number of available actions
print(env.unwrapped.get_action_meanings()) #description of actions


state = env.reset()

for i in range(10):
    
    #initialize lists to keep track of rewards, states and actions
    r = []
    observations =[]
    actions =[]
    
    state = env.reset() # initialize the game by resetting. we assign the initial state to variable state. Every time the game finishes you need to reset the game.
    
    while True:
        action = np.random.randint(n_actions) #choose random action
        next_state, reward, done, info = env.step(action) #take the actions and see the next state, reward, if the game is finished or not and some info about the game.
        
        #keep track of rewards, states and actions
        r.append(reward)
        observations.append(state)
        actions.append(action)
        
        state = next_state #assign next_state to state
        
        if done:
            break

In [34]:
state_size = env.observation_space.shape[0]

In [36]:
env.reward_range

(-inf, inf)

In [ ]:
tf.reset_default_graph()

state = tf.placeholder(tf.float32, shape=(None, state_size))
actions = tf.placeholder(tf.int32, shape=(None, 1))
rewards = tf.placeholder(tf.int32, shape=(None, 1))

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                              activation=tf.nn.relu, kernel_initializer = tf.contrib.layers.xavier_initializer())
    #print(hidden1)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                              activation=tf.nn.relu)
    
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    #y_proba = tf.nn.softmax(logits)

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits) #for this fucntion you do not need to use one hot endcoding
    loss = tf.reduce_mean(xentropy, name="loss")
    


with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
    